In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

napady
===
Vykreslenie df(xk) pre konstantny krok tak ze zobrazis # iteraciu na x osi a df(ci co) na y a bude pekne vidiet kt konverguju a kt nie 

In [ ]:
train_data = 'https://github.com/adam-213/MVO2022/raw/main/Data/HDP.txt'
test_data = 'https://github.com/adam-213/MVO2022/raw/main/Data/HDP_test.txt'

train = pd.read_csv(train_data)
test = pd.read_csv(test_data)
#M1 == x1, P == x2 , Y == y

In [ ]:
n = int(1e+7)
# 7 == 7min, 8 == 1h10m, 9 == 13h, 10 == 125h
eps = 1e-2
c = 1e-7
pwd = !pwd
root = "/".join(pwd[0].split('/')[:-1])

generate = True # try startpoints and other parameters and save results to pickle files
#turn off to just load them

In [ ]:
train_x = train.iloc[:,[2,3]].to_numpy() # x1,x2 -> np.array
X = np.hstack([np.ones([train.shape[0],1]),train_x]) # add column vector to the front of the matrix
b = np.array([1,1,1]) #default coefficients for vector of variables beta. 
y = train.iloc[:,1].to_numpy() #actual data from real world 


In [ ]:
def mns(b):
    """L2_norm^2"""
    return b.T@X.T@X@b -2*y.T@X@b + y.T@y 

def df_mns(b):
    """derivative of L2_norm^2"""
    return 2*(X.T@X@b) - 2*(X.T@y)

def df2_mns(b):
    return 2*(X.T@X) 


In [ ]:
def grad_const(f, df, x0, eps=1e-2, n=1000000, c=1e-5):
    """Gradient method with constant step"""
    xk = x0
    i = 0 
    
    for i in tqdm(range(n)):
        if np.isnan(xk[0]):
            return None,None
        if i != 0 and (xk == x_pred).all():
            return None,None
        if np.linalg.norm(df(xk)) < eps:
            break
        x_pred = xk 
        sk = -df(xk)
        xk = xk + (c * sk) 

    return xk, i

In [ ]:
def grad_backtracking(f, df, x0, eps=1e-2, n=1000000, alpha=0.1, delta=0.5, c=1):
    """semi-optimal step gradient method with backtracking for step length"""
    xk = x0
    i = 0
    for i in tqdm(range(n)):
        if np.isnan(xk[0]):
            return None,None
        if i != 0 and (xk == x_pred).all():
            return None,None
        if np.linalg.norm(df(xk)) < eps:
            break
        s = -df(xk)
        while np.linalg.norm(f(xk + c * s)) > np.linalg.norm(f(xk) + c * alpha * s.T @ df(xk)):
            c = delta * c
        x_pred = xk    
        xk = xk + c * s
        

    return xk,i

In [ ]:
def cca_opt_basil(f, df, x0, eps=1e-8, n=10000):
    """Barzilai & Borwein method of semi-optimal step based on https://pages.cs.wisc.edu/~swright/726/handouts/barzilai-borwein.pdf"""
    x_k = x0.copy()
    x_k_1 = eps
    g_k_1 = eps
    i = 0
    for i in tqdm(range(n)):
        if np.isnan(xk[0]):
            return None,None
        if i != 0 and (x_k == x_pred).all():
            return None,None
        if np.linalg.norm(df(x_k)) < eps:
            break
        g_k = df(x_k)
        delta_x = x_k - x_k_1
        delta_g = g_k - g_k_1
        alpha_k = np.dot(delta_x, delta_x) / np.dot(delta_x, delta_g)
        # s = alpha_k * np.eye(x_k.shape[0])
        x_k_1 = x_k
        g_k_1 = g_k
        x_pred = x_k 
        x_k = x_k - alpha_k * g_k

    return x_k, i + 1

In [ ]:
gold = 1 / (((np.sqrt(5) + 1)) / 2)
def gold_section(f, df, a0, b0, eps, n):
    """Golden section search for cauchy's method to determine the optimal step"""
    a, b = a0, b0
    c = gold * a + (1 - gold) * b
    d = (1 - gold) * a + gold * b
    for i in range(n):
        if f(c) < f(d):
            b = d
            d = c
            c = gold * a + (1 - gold) * b

        else:
            a = c
            c = d
            d = (1 - gold) * a + gold * b
        if b - a < eps:
            break
    if a == 0:
        return b
    return a

def cauchy(f, df, x0, eps=1e-8, n=10000):
    """Cauchy's gradient method with optimal step based on golden-section search"""
    xk = x0.copy()
    i = 0
    for i in range(n):
        if np.isnan(xk[0]):
            return None,None
        if i != 0 and (xk == x_pred).all():
            return None,None
        if np.linalg.norm(df(xk)) < eps:
            break
        s = -df(xk)
        g = lambda l: f(xk + l * s)
        c = gold_section(g, None, 0, 1, eps, 1000)
        x_pred = xk
        xk = xk + c * s
        # print(s)
    return xk, i + 1

In [ ]:
#Experimenting with step lengths and start positions on gradient method with constant step
#Writing serialized results to pickle file - can be read by pickle.load(<file>)
#Because this takes a long time to run ~1h for each block

#Constant step
if generate:
    cs = [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9] #step length
    bs = [[0,0,0],[1,1,1],[-1,-1,-1],[1,7,-7],[0.1,0.9,1.1]] #starting point

    const_grad_results = []

    for c in cs:
        for b in bs:
            b = np.array(b)
            res = grad_const(mns,df_mns,b,eps,n,c)
            const_grad_results.append((res,b,c))

    with open(root + "/Data/grad_const.pickle","wb") as c:
        pickle.dump(const_grad_results,c)
else:
    with open(root + "/Data/grad_const.pickle",'rb') as c:
        const_grad_results = pickle.load(c)


In [ ]:
#Semi-optimal step with backtracking
if generate:
    #TODO FILL
    alphas = [0.1,0.51]
    deltas = [0.2,0.9]
    cs = [2,1,1e-1]
    bs = [[0,0,0],[1,1,1],[-1,-1,-1],[0.1,0.9,1.1]]
    backtracking_results = []

    for alpha in alphas:
        for delta in deltas:
            for c in cs:
                for b in bs:
                    b = np.array(b)
                    res = grad_backtracking(mns,df_mns,b,eps,n,alpha,delta,int(c))
                    backtracking_results.append((res,b,c,alpha,delta))
    
    with open(root + "/Data/backtracking.pickle","wb") as c:
        pickle.dump(const_grad_results,c)
else:
    with open(root + "/Data/backtracking.pickle",'rb') as c:
        backtracking_results = pickle.load(c)

In [ ]:
#Brasil semi-optimal step
if generate:
    #TODO FILL
    bs = [[0,0,0],[1,1,1],[-1,-1,-1],[0.1,0.9,1.1]]
    barbor_results = []
    for b in bs:
        b = np.array(b)
        res = cca_opt_basil(mns,df_mns,b,eps,n)
        barbor_results.append(res,b)

    with open(root + "/Data/BarBor.pickle",'wb') as c:
        pickle.dump(barbor_results,c)

else:
    with open(root + "/Data/BarBor.pickle",'rb') as c:
        barbor_results = pickle.load(c)

In [ ]:
#golden_cauchy
if generate:
    bs = [[0,0,0],[1,1,1],[-1,-1,-1],[1,7,-7],[0.1,0.9,1.1]]
    cauchy_results = []
    for b in bs:
        b = np.array(b)
        res = cauchy(mns,df_mns,b,eps,n)
        cauchy_results.append(res,b)

    with open(root + "/Data/cauchy.pickle",'wb') as c:
        pickle.dump(cauchy_results,c))

else:
    with open(root + "/Data/cauchy.pickle",'rb') as c:
        cauchy_results = pickle.load(c)  


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c876a9c3-8d90-46cc-9c0c-98291fab2bc5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>